# Arihant Shashank (Big data and aws engineer)
## CGS_Task_1

#### Problem which is understood:
This problem is basically to to get the streamed data which is in json format and join it with static hive table and create a new hive table


Assumptions:

1) I have used all the data set creation through python only , not calling any file specifically to make sure everything is visible in one code


2) Kafka setup i dont have , so kindly consider my kafka data as static data here 


3) Some optimization i will be using because the data of hive will be smaller as compared to kafka data so hive data needs to be broadcasted


4) Kafka i have take 10 records and hive i have taken 12 records

5) As i dont have any scheduler, so i will be shceduling it in aws glue or autosys based to event trigger of kafka

In [8]:
# Initiating spark
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark = SparkSession.builder.appName("cgs_task").getOrCreate()
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pandas as pd

### Creating the json table data

In [9]:

# Lets assume we have a data landing from the kafka into a specific s3 or hdfs location with below format
# kakfa json 
cutomer_info=cutomer_info=[{
  "cust_id": 101,
  "age": 20,
  "gender": "male",
  "City": "hyd"
},
{
  "cust_id": 102,
  "age": 21,
  "gender": "female",
  "City": "blr"
},
{
  "cust_id": 103,
  "age": 22,
  "gender": "male",
  "City": "kol"
},
{
  "cust_id": 104,
  "age": 22,
  "gender": "male",
  "City": "ixr"
},
{
  "cust_id": 105,
  "age": 23,
  "gender": "male",
  "City": "del"
},
{
  "cust_id": 106,
  "age": 25,
  "gender": "female",
  "City": "mas"
},
{
  "cust_id": 107,
  "age": 26,
  "gender": "female",
  "City": "mum"
},
{
  "cust_id": 108,
  "age": 29,
  "gender": "female",
  "City": "pune"
},
{
  "cust_id": 109,
  "age": 30,
  "gender": "female",
  "City": "dubai"
},
{
  "cust_id": 110,
  "age": 704,
  "gender": "female",
  "City": "texas"
}]



### Creating the hive table data

In [10]:
from pyspark.sql.functions import broadcast

schema= StructType([StructField("email_id",StringType(), True), \
                    StructField("cust_id",IntegerType(),True)
                   ])

hive_data =data2= [("xyz@gmail.com",102),
    ("abc@gmail.com",103),
    ("alpha@ymail.com",104),
    ("asd@yahoo.com",105),
    ("qwerty@outlook.com",106)]

In [11]:
def read_json_data(cutomer_info):
    """this function will read the json data defined above by converting into rdd
    cutomer_info is the json dataset"""
    custi_info_rdd=sc.parallelize(cutomer_info)
    customer_info_read=spark.read.json(custi_info_rdd)
    return customer_info_read

In [12]:
def read_hive_data(hive_data,schema):
    """reading the data for hive hive_data= dataset which i created above,
    schema is the schema i defined above for this schema"""
    hive_table = spark.createDataFrame(data=hive_data,schema=schema)
    return hive_table
    

#### Question :The ask is to write Spark code which will read both the sources in a recurring scheduled job(daily/weekly etc) and write out .csv 
As i dont have any scheduler, so i will be shceduling it in aws glue or autosys based to event trigger of kafka


In [20]:
def full_dataset(broadcast_hive_table,customer_info_read):
    """broadcast_hive_table: this will be hive table
        customer_info_read: this is json df
        this function will create the overall dataset"""
    full_dataset_data=customer_info_read.join(broadcast_hive_table,customer_info_read.cust_id==broadcast_hive_table.cust_id,"left").\
                    select("email_id",
                          customer_info_read["cust_id"],
                          "City",
                          "age",
                          "gender")
    return full_dataset_data

# Main() function

In [21]:
if __name__ == "__main__":
    hive_table = spark.createDataFrame(data=hive_data,schema=schema)
    broadcast_hive_table=broadcast(hive_table)
    # here i am brodcasting the hive table
    customer_info_read= read_json_data(cutomer_info)
    full_dataset=full_dataset(broadcast_hive_table,customer_info_read)
    full_dataset.show()
    full_dataset.toPandas().to_csv("csg_probelm_pyspark.csv", header=True)


+------------------+-------+-----+---+------+
|          email_id|cust_id| City|age|gender|
+------------------+-------+-----+---+------+
|              null|    101|  hyd| 20|  male|
|     xyz@gmail.com|    102|  blr| 21|female|
|     abc@gmail.com|    103|  kol| 22|  male|
|   alpha@ymail.com|    104|  ixr| 22|  male|
|     asd@yahoo.com|    105|  del| 23|  male|
|qwerty@outlook.com|    106|  mas| 25|female|
|              null|    107|  mum| 26|female|
|              null|    108| pune| 29|female|
|              null|    109|dubai| 30|female|
|              null|    110|texas|704|female|
+------------------+-------+-----+---+------+

